In [1]:
# import spacy
# from spacytextblob.spacytextblob import SpacyTextBlob

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
41,application_1671409217564_0063,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
# nlp = spacy.load('en_core_web_sm')
# nlp.add_pipe('spacytextblob')

In [3]:
# text = 'I had a okayish day'
# doc = nlp(text)
# doc._.blob.polarity

In [1]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *
from operator import add

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
45,application_1671409217564_0067,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
filePath = 'abfs://dda-2022-12-15t21-04-18-212z@ddasta.dfs.core.windows.net/reddit/2020/'
years = ['RC_2020-10.json', 'RC_2020-11.json', 'RC_2020-12.json']
# years = ['RC_2020-10.json']
jsonFiles = []

for y in years:
    p = filePath+y
    jsonFiles.append(p)


schema = StructType([
    StructField('subreddit', StringType(), nullable=True),
    StructField('body', StringType(), nullable=True)
])

df = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)

# Loop through the list of JSON files and read them
for file in jsonFiles:
    temp_df = spark.read.json(file, schema=schema)
    df = df.union(temp_df)

# Print the schema of the DataFrame
df.printSchema()

root
 |-- subreddit: string (nullable = true)
 |-- body: string (nullable = true)

In [3]:
from pyspark.sql.functions import udf

In [4]:
import spacy

In [5]:
# from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("spacytextblob")

In [6]:
def sentimentScore (subreddit, body):
    if (not nlp.has_pipe("spacytextblob")):
        from spacytextblob.spacytextblob import SpacyTextBlob
        nlp.add_pipe("spacytextblob")
    
    doc = nlp(body)
    polarity = doc._.blob.polarity
    return polarity

sentimentScoreUDF = udf(sentimentScore)

In [7]:
# df.take(1)

In [8]:
filteredSoccerDf = df.filter(df.body != '[deleted]').filter(df.subreddit == 'soccer')
filteredNbaDf = df.filter(df.body != '[deleted]').filter(df.subreddit == 'nba')

In [9]:
sentimentScoreSoccer = filteredSoccerDf.withColumn("sentiment", sentimentScoreUDF(df.subreddit, df.body))
sentimentScoreNba = filteredNbaDf.withColumn("sentiment", sentimentScoreUDF(df.subreddit, df.body))

### Removing all neutrals, since there are alot of posts with 1 word comment, or links which are neutral

In [12]:
sentimentScoreSoccerWithoutNeutral = sentimentScoreSoccer.filter(sentimentScoreSoccer.sentiment != '0.0').cache()
sentimentScoreNbaWithoutNeutral = sentimentScoreNba.filter(sentimentScoreNba.sentiment != '0.0').cache()

In [26]:
sentimentScoreSoccerWithoutNeutral.take(1)

[Row(subreddit='soccer', body="If this isn't sarcasm ol mate Howard is batting above his average.", sentiment='-0.075')]

In [27]:
sentimentScoreNbaWithoutNeutral.take(1)

[Row(subreddit='nba', body='lmao', sentiment='0.6')]

In [36]:
type(sentimentScoreSoccerWithoutNeutral)

<class 'pyspark.sql.dataframe.DataFrame'>

In [37]:
sentimentScoreSoccerWithoutNeutral.printSchema()

root
 |-- subreddit: string (nullable = true)
 |-- body: string (nullable = true)
 |-- sentiment: string (nullable = true)

In [43]:
totalSentimentSoccer = sentimentScoreSoccerWithoutNeutral.select(sum(sentimentScoreSoccerWithoutNeutral.sentiment)).first()[0]

In [44]:
totalSentimentNba = sentimentScoreNbaWithoutNeutral.select(sum(sentimentScoreNbaWithoutNeutral.sentiment)).first()[0]

In [45]:
totalSentimentSoccer

166674.4246234904

In [46]:
totalSentimentNba

223171.5416214871

# Analysis

As we can see people from the Nba subreddit are much more positive than the people from r/soccer

In [10]:
sentimentScoreSoccerOnlyNegative = sentimentScoreSoccer.filter(sentimentScoreSoccer.sentiment < '0.0').cache()
sentimentScoreNbaOnlyNegative = sentimentScoreNba.filter(sentimentScoreNba.sentiment < '0.0').cache()

In [11]:
totalNegativeSentimentSoccer = sentimentScoreSoccerOnlyNegative.select(sum(sentimentScoreSoccerOnlyNegative.sentiment)).first()[0]
totalNegativeSentimentNba = sentimentScoreNbaOnlyNegative.select(sum(sentimentScoreNbaOnlyNegative.sentiment)).first()[0]

In [12]:
print(totalNegativeSentimentSoccer)
print(totalNegativeSentimentNba)

-129429.56723046652
-128497.50436319303

In [10]:
sentimentScoreSoccerForAvg = sentimentScoreSoccer.filter(sentimentScoreSoccer.sentiment != '0.0').cache()
sentimentScoreNbaForAvg = sentimentScoreNba.filter(sentimentScoreNba.sentiment != '0.0').cache()

In [11]:
averageSentimentSoccer = sentimentScoreSoccerForAvg.select(avg(sentimentScoreSoccerForAvg.sentiment)).first()[0]
averageSentimentNba = sentimentScoreNbaForAvg.select(avg(sentimentScoreNbaForAvg.sentiment)).first()[0]

### On average, as we can see from down below, the sentiment on both the subreddits (excluding the neutrals) are almost the same

In [12]:
averageSentimentSoccer

0.11253664559864221

In [13]:
averageSentimentNba

0.13772603441706835